# Create spreadsheet(s) for:
- Number of impaired days
- Percent volume of Baseline - Scenario < 0.2 mg/l
- Percent volume per run of DO < 2 mg/l
- Percent volume per run of DO < 5 mg/l

In [ ]:
## need to re-run batch script to include reference DO

In [ ]:
np_operator='min'
model_var='DOXG'
processed_netcdf_dir = pathlib.Path(ssm['paths']['processed_output'])/model_var
# get list of run sub-directories in processed netcdf directory
dir_list = os.listdir(processed_netcdf_dir)

In [ ]:
MinBottomDO_full={}
for run_dir in dir_list:
    try: 
        run_file=processed_netcdf_dir/run_dir/'bottom'/f'daily_{np_operator}_{model_var}_bottom.nc'
        with xarray.open_dataset(run_file) as ds:
            print([*ds])
            MinBottomDO_full[run_dir]=ds[f'{model_var}_daily_{np_operator}_bottom']
            print(MinBottomDO_full[run_dir].shape)
    except FileNotFoundError:
        print(f'File Not Found: {run_file}')
        # create empty placeholder for now
        MinBottomDO_full[run_dir]=np.ones((361, 16012))*9999

In [ ]:
nnodes = len(DO_thresh)
ndays=361 # first 5-days are consisdered spin-up and have been removed
ImpairedDays={}
# create array of DO_threshold values to determine number of days of noncompliance
DO_thresh2D = np.ones((nnodes,ndays))*np.array(DO_thresh).reshape(nnodes,1)
# (7494,361) x (7494,1) => element-wise multiplication 

In [ ]:
Impaired={}
for run_type in dir_list:
    print(run_type)
    Impaired[run_type] = MinBottDO[run_type]<=DO_thresh2D.transpose() #361x7494 (nodes x time)
    ImpairedDays[run_type]=Impaired[run_type].sum(axis=0) #7494 (nodes)

In [ ]:
TotalImpaired={}
basins = gdf[['node_id','Basin']].groupby('Basin').count().index.to_list()
for run_type in dir_list:
    TotalImpaired[run_type]={}
    for basin in basins:
        TotalImpaired[run_type][basin]=np.array(ImpairedDays[run_type])[
            (gdf['Basin']==basin) &
            (gdf['included_i']==1)
        ].sum()

In [ ]:
# Total number of Impaired days by basin and run
TotalImpaired_df=pandas.DataFrame(TotalImpaired)
TotalImpaired_df=TotalImpaired_df.rename(columns=ssm['run_information']['run_tag'])
TotalImpaired_df = TotalImpaired_df.reindex(columns=['Baseline','Reference','1b','1c','1d','1e','2a','2b'])



In [ ]:
import openpyxl
excel_output_path = ssm['paths']['processed_output']
with pandas.ExcelWriter(pathlib.Path(excel_output_path)/'ImpairedDays.xlsx', mode='w') as writer:  
    TotalImpaired_df.to_excel(writer, sheet_name='Total_Impaired_Days')